In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.model_selection import train_test_split 

In [2]:
from google.colab import files
files.upload()

Saving test_lAUu6dG.csv to test_lAUu6dG.csv
Saving train_ctrUa4K.csv to train_ctrUa4K.csv


{'test_lAUu6dG.csv': b'Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area\r\nLP001015,Male,Yes,0,Graduate,No,5720,0,110,360,1,Urban\r\nLP001022,Male,Yes,1,Graduate,No,3076,1500,126,360,1,Urban\r\nLP001031,Male,Yes,2,Graduate,No,5000,1800,208,360,1,Urban\r\nLP001035,Male,Yes,2,Graduate,No,2340,2546,100,360,,Urban\r\nLP001051,Male,No,0,Not Graduate,No,3276,0,78,360,1,Urban\r\nLP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152,360,1,Urban\r\nLP001055,Female,No,1,Not Graduate,No,2226,0,59,360,1,Semiurban\r\nLP001056,Male,Yes,2,Not Graduate,No,3881,0,147,360,0,Rural\r\nLP001059,Male,Yes,2,Graduate,,13633,0,280,240,1,Urban\r\nLP001067,Male,No,0,Not Graduate,No,2400,2400,123,360,1,Semiurban\r\nLP001078,Male,No,0,Not Graduate,No,3091,0,90,360,1,Urban\r\nLP001082,Male,Yes,1,Graduate,,2185,1516,162,360,1,Semiurban\r\nLP001083,Male,No,3+,Graduate,No,4166,0,40,180,,Urban\r\nLP001094,Male,Yes,2,Graduat

In [5]:
import h2o
h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpa6c2v6js
  JVM stdout: /tmp/tmpa6c2v6js/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpa6c2v6js/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.5
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_r2awn8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [9]:
train=pd.read_csv('/content/train_ctrUa4K.csv')
test=pd.read_csv('/content/test_lAUu6dG.csv')

In [10]:
train=train.drop(labels='Loan_ID', axis=1)
test=test.drop(labels='Loan_ID', axis=1)

In [11]:
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [12]:
traindf=h2o.H2OFrame(train)
testdf=h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
traindf['Gender']=traindf['Gender'].asfactor()
traindf['Married']=traindf['Married'].asfactor()
traindf['Dependents']=traindf['Dependents'].asfactor()
traindf['Education']=traindf['Education'].asfactor()
traindf['Self_Employed']=traindf['Self_Employed'].asfactor()
traindf['Property_Area']=traindf['Property_Area'].asfactor()




testdf['Gender']=testdf['Gender'].asfactor()
testdf['Married']=testdf['Married'].asfactor()
testdf['Dependents']=testdf['Dependents'].asfactor()
testdf['Education']=testdf['Education'].asfactor()
testdf['Self_Employed']=testdf['Self_Employed'].asfactor()
testdf['Property_Area']=testdf['Property_Area'].asfactor()



In [15]:
predictors = traindf.columns[:-1]
response = "Loan_Status"

In [16]:
train, test = traindf.split_frame(ratios = [0.8], seed = 1234)

In [18]:
glm_model = H2OGeneralizedLinearEstimator(family= "binomial",lambda_ = 0,compute_p_values = True)
glm_model.train(predictors, response, training_frame= train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [19]:
predict = glm_model.predict(testdf)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [20]:
predict.head()

predict,N,Y,StdErr
Y,0.144647,0.855353,0.298849
Y,0.264343,0.735657,0.336728
Y,0.124001,0.875999,0.407343
Y,0.187473,0.812527,0.402576
Y,0.300735,0.699265,0.387809
Y,0.149212,0.850788,0.550519
Y,0.298247,0.701753,0.485202
N,0.930066,0.0699342,0.598305
Y,0.143045,0.856955,0.650687
Y,0.182521,0.817479,0.395552


In [22]:
perf = glm_model.model_performance(test)
perf


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.1852806437366948
RMSE: 0.4304423814364645
LogLoss: 0.564870522858373
Null degrees of freedom: 120
Residual degrees of freedom: 104
Null deviance: 162.72559349947062
Residual deviance: 136.69866653172627
AIC: 170.69866653172627
AUC: 0.7070175438596491
AUCPR: 0.7533862360221786
Gini: 0.4140350877192982

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5024007941032466: 


,,N,Y,Error,Rate
0,N,18.0,27.0,0.6,(27.0/45.0)
1,Y,1.0,75.0,0.0132,(1.0/76.0)
2,Total,19.0,102.0,0.2314,(28.0/121.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.502401,0.842697,101.0
1,max f2,0.502401,0.923645,101.0
2,max f0point5,0.502401,0.774793,101.0
3,max accuracy,0.502401,0.768595,101.0
4,max precision,0.913147,0.882353,16.0
5,max recall,0.075090,1.000000,114.0
6,max specificity,0.964762,0.977778,0.0
7,max absolute_mcc,0.502401,0.513890,101.0
8,max min_per_class_accuracy,0.754930,0.622222,64.0
9,max mean_per_class_accuracy,0.502401,0.693421,101.0



Gains/Lift Table: Avg response rate: 62.81 %, avg score: 47.94 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.016529,1.245538,0.796053,0.796053,0.500000,1.386404,0.500000,1.386404,0.013158,0.013158,-20.394737,-20.394737,-0.009064
1,2,0.024793,1.022422,0.000000,0.530702,0.000000,1.046010,0.333333,1.272939,0.000000,0.013158,-100.000000,-46.929825,-0.031287
2,3,0.033058,0.985314,0.000000,0.398026,0.000000,0.987040,0.250000,1.201464,0.000000,0.013158,-100.000000,-60.197368,-0.053509
3,4,0.041322,0.953402,1.592105,0.636842,1.000000,0.984163,0.400000,1.158004,0.013158,0.026316,59.210526,-36.315789,-0.040351
4,5,0.057851,0.940290,0.796053,0.682331,0.500000,0.943001,0.428571,1.096574,0.013158,0.039474,-20.394737,-31.766917,-0.049415
5,6,0.107438,0.655876,0.265351,0.489879,0.166667,0.713968,0.307692,0.919987,0.013158,0.052632,-73.464912,-51.012146,-0.147368
6,7,0.157025,0.586235,0.530702,0.502770,0.333333,0.617915,0.315789,0.824596,0.026316,0.078947,-46.929825,-49.722992,-0.209942
7,8,0.206612,0.550345,0.265351,0.445789,0.166667,0.565286,0.280000,0.762361,0.013158,0.092105,-73.464912,-55.421053,-0.307895
8,9,0.305785,0.512287,0.663377,0.516358,0.416667,0.532064,0.324324,0.687670,0.065789,0.157895,-33.662281,-48.364154,-0.397661
9,10,0.404959,0.481971,1.194079,0.682331,0.750000,0.494111,0.428571,0.640268,0.118421,0.276316,19.407895,-31.766917,-0.345906
